In [1]:
import deepspeed, torch

import importlib, os

from models.deepseek import model
importlib.reload(model)

from models.deepseek import train
importlib.reload(train)

from models.deepseek import data
importlib.reload(data)

/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/ubuntu/projects/llm-dev/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'models.deepseek.data' from '/home/ubuntu/projects/llm-models/llm-models/models/deepseek/data.py'>

In [2]:
scale = 2
model_args = model.ModelArgs(expert_type=model.ExpertType(2),
                             dim=128*scale, inter_dim=341*scale, moe_inter_dim=64*scale,
                             kv_lora_rank=32*scale, qk_nope_head_dim=8*scale, qk_rope_head_dim=4*scale, 
                             v_head_dim=8*scale, 
                             n_routed_experts=32, n_layers=4*scale,
                                original_seq_len=data.CONTEXT_LENGTH)


training_args = train.TrainingArgs(
    model_args=model_args,
    batch_size=4,
    gradient_accumulation_steps=16,
    max_steps=100000,
    learning_rate=3e-4,
    checkpoint_dir="./checkpoints",
    data_dir="./data"
)


trainer = train.Trainer(training_args)

Total parameters: 108,939,520
Trainable parameters: 108,939,520
Loading *tokenized* TinyStories dataset split from: './processed_tinystories/train'...


/home/ubuntu/projects/llm-models/llm-models/models/deepseek/train.py:206: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.args.use_amp


Tokenized dataset loaded. Total chunks: 459757
Loading *tokenized* TinyStories dataset split from: './processed_tinystories/validation'...
Tokenized dataset loaded. Total chunks: 4619


In [3]:

ckpt_path = '/home/ubuntu/projects/llm-models/ckpts-expert/best.pt'
trainer.load_checkpoint(ckpt_path)

Loaded checkpoint from /home/ubuntu/projects/llm-models/ckpts-expert/best.pt
Resuming from step 7500


In [29]:
trainer.model.state_dict()

OrderedDict([('embed.weight',
              tensor([[-0.0464, -0.0957, -0.0117,  ..., -0.0981, -0.0713, -0.0364],
                      [ 0.0137,  0.0903, -0.0547,  ...,  0.0732,  0.0649, -0.0469],
                      [ 0.0903, -0.0075, -0.0234,  ...,  0.0659, -0.0417,  0.0059],
                      ...,
                      [-0.0452,  0.0146,  0.0239,  ...,  0.0339, -0.0058,  0.0510],
                      [-0.0020,  0.0105,  0.0178,  ..., -0.0884,  0.0287, -0.0247],
                      [ 0.0212,  0.1001, -0.1221,  ...,  0.1147,  0.0317, -0.0811]],
                     device='cuda:0')),
             ('layers.0.attn.wq.weight',
              tensor([[-0.0166,  0.0613, -0.0420,  ...,  0.0535, -0.0132, -0.0400],
                      [ 0.0186, -0.0596,  0.0192,  ..., -0.0031, -0.0327,  0.0469],
                      [ 0.0129,  0.0280, -0.0342,  ...,  0.0126, -0.0588, -0.0459],
                      ...,
                      [-0.0413, -0.0388, -0.0054,  ...,  0.0366,  0.0625, -0.0

In [39]:
para_name1 = 'layers.1.ffn.experts.deepspeed_moe.experts.deepspeed_experts.0._orig_mod.block.attn.learnable_k_cache'
para_name2 = 'layers.1.ffn.experts.deepspeed_moe.experts.deepspeed_experts.1._orig_mod.block.attn.learnable_k_cache'
trainer.model.state_dict()[para_name1].shape

torch.Size([1, 32, 16, 24])

In [41]:
# tensor_k_value = trainer.model.state_dict()[para_name1]
# torch.einsum('bsle,bsre->bslr', tensor_k_value, tensor_k_value)

In [42]:
tensor_k_value = trainer.model.state_dict()[para_name1]
torch.corrcoef(tensor_k_value[0, :, 1, :])

tensor([[ 1.0000,  0.3613,  0.0342,  ...,  0.3223,  0.2793, -0.2305],
        [ 0.3613,  1.0000,  0.1465,  ...,  0.2988,  0.1494, -0.1895],
        [ 0.0342,  0.1465,  1.0000,  ..., -0.1309,  0.2119, -0.0742],
        ...,
        [ 0.3223,  0.3008, -0.1309,  ...,  1.0000, -0.1543, -0.0977],
        [ 0.2793,  0.1484,  0.2119,  ..., -0.1543,  0.9961, -0.0693],
        [-0.2305, -0.1895, -0.0742,  ..., -0.0972, -0.0693,  1.0000]],
       device='cuda:0')

In [43]:
tensor_k_value = trainer.model.state_dict()[para_name2]
torch.corrcoef(tensor_k_value[0, :, 1, :])

tensor([[ 1.0000,  0.2070, -0.1963,  ...,  0.3496, -0.1196, -0.1465],
        [ 0.2070,  1.0000,  0.0488,  ...,  0.2119,  0.2061, -0.1885],
        [-0.1953,  0.0488,  1.0000,  ..., -0.1475, -0.0125, -0.0874],
        ...,
        [ 0.3516,  0.2119, -0.1484,  ...,  1.0000, -0.1836,  0.0679],
        [-0.1196,  0.2061, -0.0125,  ..., -0.1826,  1.0000, -0.0025],
        [-0.1465, -0.1885, -0.0874,  ...,  0.0679, -0.0025,  1.0000]],
       device='cuda:0')

In [44]:
tensor_k_value

tensor([[[[-0.0327, -0.0311, -0.0157,  ...,  0.0182, -0.0403,  0.0386],
          [ 0.0186, -0.0269, -0.0079,  ..., -0.0156, -0.0435,  0.0339],
          [-0.0481, -0.0071, -0.0167,  ..., -0.0101, -0.0023, -0.0413],
          ...,
          [ 0.0312, -0.0187, -0.0176,  ...,  0.0034,  0.0330,  0.0282],
          [-0.0244,  0.0383,  0.0084,  ..., -0.0339,  0.0286, -0.0103],
          [-0.0222, -0.0155, -0.0430,  ...,  0.0159,  0.0352,  0.0260]],

         [[-0.0435,  0.0015,  0.0060,  ...,  0.0166, -0.0315, -0.0479],
          [-0.0403, -0.0294, -0.0386,  ..., -0.0422, -0.0205,  0.0391],
          [-0.0021, -0.0383,  0.0503,  ...,  0.0231, -0.0410, -0.0036],
          ...,
          [-0.0035, -0.0153, -0.0312,  ...,  0.0315, -0.0400, -0.0334],
          [-0.0244, -0.0031,  0.0182,  ...,  0.0190, -0.0162, -0.0082],
          [-0.0126,  0.0386, -0.0320,  ..., -0.0159, -0.0155, -0.0452]],

         [[ 0.0417, -0.0311,  0.0004,  ...,  0.0312,  0.0216, -0.0250],
          [ 0.0280,  0.0010, -

In [36]:
tensor_k_value[0, :, 0, :].shape

torch.Size([32, 24])

In [16]:
ckpt_path = '/home/ubuntu/projects/llm-models/checkpoints/best.pt'
ckpt = torch.load(ckpt_path, map_location="cpu")

In [46]:
next(trainer.model.parameters()).device

device(type='cuda', index=0)

In [48]:
class ActivationMonitor:
    def __init__(self, model):
        self.hooks = []
        for name, module in model.named_modules():
            # Filter for layers you care about (e.g., Norm, Linear, Attention)
            # if isinstance(module, (torch.nn.Linear, torch.nn.LayerNorm)):
            # if 'layers.7.attn.kv_norm' in name:
            #     hook = module.register_full_backward_hook(self.make_hook(name))
            #     self.hooks.append(hook)
            if 'experts.deepspeed_moe.experts.deepspeed_experts' in name:
                hook = module.register_forward_hook(self.make_forward_hook(name))
                self.hooks.append(hook)

    def make_hook(self, name):
        def hook(module, grad_input, grad_output):
            # grad_output[0] is the gradient of the loss w.r.t. the output of this layer
            g = grad_output[0]
            # print(g)
            if g is not None:
                # print(f'{name}: max: {torch.max(g)}, min: {torch.min(g)}')
                print(f"[{name}] Output Grad -> Min: {g.min():.4e} | Max: {g.max():.4e} | Mean: {g.mean():.4e}")
                if torch.isnan(g).any():
                    print(f"!!! NAN DETECTED IN {name} Output GRADIENT !!!")
                    
            g = grad_input[0]
            # print(g)
            if g is not None:
                # print(f'{name}: max: {torch.max(g)}, min: {torch.min(g)}')
                print(f"[{name}] Input Grad -> Min: {g.min():.4e} | Max: {g.max():.4e} | Mean: {g.mean():.4e}")
                if torch.isnan(g).any():
                    print(f"!!! NAN DETECTED IN {name} Input GRADIENT !!!")
                    
        return hook

    def make_forward_hook(self, name):
        def hook(module, act_input, act_output):
            # grad_output[0] is the gradient of the loss w.r.t. the output of this layer
            g = act_output[0]
            # print(g)
            if g is not None:
                # print(f'{name}: max: {torch.max(g)}, min: {torch.min(g)}')
                print(f"[{name}] Output Activation -> Min: {g.min():.4e} | Max: {g.max():.4e} | Mean: {g.mean():.4e}")
                if torch.isnan(g).any():
                    print(f"!!! NAN DETECTED IN {name} Output GRADIENT !!!")
                
            g = act_input[0]
            # print(g)
            if g is not None and name != 'embed':
                # print(f'{name}: max: {torch.max(g)}, min: {torch.min(g)}')
                print(f"[{name}] Input Activation -> Min: {g.min():.4e} | Max: {g.max():.4e} | Mean: {g.mean():.4e}")
                if torch.isnan(g).any():
                    print(f"!!! NAN DETECTED IN {name} Input GRADIENT !!!")
                    
        return hook
        
    def remove(self):
        for h in self.hooks:
            h.remove()

In [49]:
monitor = ActivationMonitor(trainer.model)

In [4]:
for step, batch in enumerate(trainer.train_loader):
    break
    
x, y = batch
x = x.to(trainer.device, non_blocking=True)
y = y.to(trainer.device, non_blocking=True)

In [ ]:
torch.Size([32, 1, 256, 16])

In [14]:
torch.einsum('shd,qbhd->sqbh', 
             torch.squeeze(torch.zeros([1, 32, 16, 24]), 0), 
             torch.zeros([1, 256, 16, 24])).shape

torch.Size([32, 1, 256, 16])

In [5]:
output = trainer.model(x)

/home/ubuntu/projects/llm-dev/lib/python3.12/site-packages/torch/_inductor/lowering.py:1988: UserWarning: Torchinductor does not support code generation for complex operators. Performance may be worse than eager.
  warnings.warn(


In [1]:
8*1024/32

256.0

In [20]:
256/4

64.0

In [19]:
x.shape

torch.Size([4, 1024])

In [6]:
trainer.model

Transformer(
  (embed): ParallelEmbedding()
  (layers): ModuleList(
    (0): Block(
      (attn): MLA(
        (wq): ColumnParallelLinear()
        (wkv_a): Linear()
        (kv_norm): RMSNorm()
        (wkv_b): ColumnParallelLinear()
        (wo): RowParallelLinear()
      )
      (ffn): MLP(
        (w1): ColumnParallelLinear()
        (w2): RowParallelLinear()
        (w3): ColumnParallelLinear()
      )
      (ffn_norm): RMSNorm()
      (attn_norm): RMSNorm()
    )
    (1-7): 7 x Block(
      (attn): MLA(
        (wq): ColumnParallelLinear()
        (wkv_a): Linear()
        (kv_norm): RMSNorm()
        (wkv_b): ColumnParallelLinear()
        (wo): RowParallelLinear()
      )
      (ffn): MoE(
        (experts): MoE(
          (deepspeed_moe): MOELayer(
            (gate): TopKGate(
              (wg): Linear(in_features=256, out_features=32, bias=False)
            )
            (experts): Experts(
              (deepspeed_experts): ModuleList(
                (0-31): 32 x Optimiz

In [23]:
trainer.model.layers[2].ffn.experts.deepspeed_moe.experts.deepspeed_experts[0]

OptimizedModule(
  (_orig_mod): Expert(
    (block): Block(
      (attn): ExpertMLA(
        (wq): Linear()
        (wkv_a): Linear()
        (kv_norm): RMSNorm()
        (wkv_b): Linear()
        (wo): Linear()
      )
      (attn_norm): RMSNorm()
    )
  )
)

In [14]:
attention_score = trainer.model.layers[4].ffn.experts.deepspeed_moe.experts.deepspeed_experts[0]._orig_mod.block.attn.attention_score

In [7]:
attention_score.shape

torch.Size([32, 1, 256, 16])

In [10]:
attention_score = torch.softmax(attention_score, dim=0)

In [15]:
attention_score[:, :, 0, 1]

tensor([[ 0.0991],
        [ 0.0383],
        [-0.0447],
        [-0.1367],
        [ 0.0413],
        [-0.1670],
        [-0.1396],
        [ 0.0464],
        [ 0.0493],
        [ 0.0801],
        [-0.0481],
        [-0.0552],
        [-0.0219],
        [ 0.0337],
        [-0.0488],
        [ 0.0082],
        [ 0.0071],
        [-0.0723],
        [ 0.0206],
        [ 0.1279],
        [-0.0165],
        [ 0.1084],
        [-0.0264],
        [ 0.0022],
        [-0.0022],
        [-0.0381],
        [-0.0075],
        [ 0.0703],
        [ 0.0359],
        [ 0.2559],
        [ 0.0125],
        [-0.0491]], device='cuda:0')

In [23]:
 torch.softmax(attention_score[:, :, 0, 9], dim=0)

tensor([[0.0292],
        [0.0339],
        [0.0312],
        [0.0339],
        [0.0287],
        [0.0305],
        [0.0309],
        [0.0262],
        [0.0309],
        [0.0354],
        [0.0293],
        [0.0361],
        [0.0361],
        [0.0282],
        [0.0272],
        [0.0247],
        [0.0361],
        [0.0300],
        [0.0310],
        [0.0322],
        [0.0273],
        [0.0342],
        [0.0347],
        [0.0408],
        [0.0275],
        [0.0308],
        [0.0278],
        [0.0306],
        [0.0302],
        [0.0330],
        [0.0332],
        [0.0281]], device='cuda:0')

In [17]:
attention_score[:, :, 0, 1]

tensor([[ 0.0991],
        [ 0.0383],
        [-0.0447],
        [-0.1367],
        [ 0.0413],
        [-0.1670],
        [-0.1396],
        [ 0.0464],
        [ 0.0493],
        [ 0.0801],
        [-0.0481],
        [-0.0552],
        [-0.0219],
        [ 0.0337],
        [-0.0488],
        [ 0.0082],
        [ 0.0071],
        [-0.0723],
        [ 0.0206],
        [ 0.1279],
        [-0.0165],
        [ 0.1084],
        [-0.0264],
        [ 0.0022],
        [-0.0022],
        [-0.0381],
        [-0.0075],
        [ 0.0703],
        [ 0.0359],
        [ 0.2559],
        [ 0.0125],
        [-0.0491]], device='cuda:0')

In [19]:
torch.exp(torch.tensor(0.0991))


tensor(1.1016)

In [20]:
torch.exp(torch.tensor(-0.0447))


tensor(0.9570)